In [1]:
import pandas as pd

In [36]:
df = pd.read_csv('backloggd_games.csv', index_col= 0)
df.drop(['Wishlist', 'Backlogs','Playing', 'Lists'], axis = 1, inplace= True)
df.drop_duplicates(inplace= True)    ### 
df

,Title,Release_Date,Developers,Summary,Platforms,Genres,Rating,Plays,Reviews
0,Elden Ring,"Feb 25, 2022","['FromSoftware', 'Bandai Namco Entertainment']","Elden Ring is a fantasy, action and open world...","['Windows PC', 'PlayStation 4', 'Xbox One', 'P...","['Adventure', 'RPG']",4.5,21K,3K
1,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",The Legend of Zelda: Breath of the Wild is the...,"['Wii U', 'Nintendo Switch']","['Adventure', 'Puzzle']",4.4,35K,3K
2,Hades,"Dec 07, 2018",['Supergiant Games'],A rogue-lite hack and slash dungeon crawler in...,"['Windows PC', 'Mac', 'PlayStation 4', 'Xbox O...","['Adventure', 'Brawler', 'Indie', 'RPG']",4.3,25K,2.1K
3,Hollow Knight,"Feb 24, 2017",['Team Cherry'],A 2D metroidvania with an emphasis on close co...,"['Windows PC', 'Mac', 'Linux', 'Nintendo Switch']","['Adventure', 'Indie', 'Platform']",4.4,25K,2.1K
4,Undertale,"Sep 15, 2015","['tobyfox', '8-4']","A small child falls into the Underground, wher...","['Windows PC', 'Mac', 'Linux', 'PlayStation 4'...","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",4.2,32K,2.5K
...,...,...,...,...,...,...,...,...,...
59992,Trace 2,TBD,[],NaN,[],[],NaN,0,0
59993,No Country for Old Men,TBD,[],This is a game combines SIM and SRPG. Please l...,[],"['Indie', 'RPG', 'Simulator', 'Strategy']",NaN,0,0
59994,Sainth,"Apr 11, 2008",['Neo Kuriyo'],"Years after the events of ""Wraith"", a young gi...",['Windows PC'],[],NaN,1,0
59995,Dragon Spirits,"Apr 25, 2023","['FHNBHJ', 'indienova']",While constantly working on a video game witho...,['Windows PC'],['RPG'],NaN,2,0


,Title,Release_Date,Developers,Summary,Platforms,Genres,Rating,Plays,Reviews
40177,#Archery,"Mar 30, 2017",[],#Archery brings out your inner archer in a way...,['Windows PC'],"['Indie', 'Sport']",NaN,1,0
39638,#Archery,"Mar 30, 2017",[],#Archery brings out your inner archer in a way...,['Windows PC'],"['Indie', 'Sport']",NaN,1,0
24930,#Drive,"Mar 16, 2019",['Pixel Perfect Dude'],#Drive is an endless driving video game inspir...,"['Android', 'iOS', 'Nintendo Switch']","['Arcade', 'Racing', 'Simulator']",NaN,5,0
24856,#Drive,"Mar 16, 2019",['Pixel Perfect Dude'],#Drive is an endless driving video game inspir...,"['Android', 'iOS', 'Nintendo Switch']","['Arcade', 'Racing', 'Simulator']",NaN,5,0
13591,(Another) Shadow the Hedgehog in Sonic the Hed...,"Jan 18, 2019",['AsuharaMoon'],A ROM hack/mod for Sonic the Hedgehog which ch...,['Sega Mega Drive/Genesis'],['Platform'],NaN,2,0
...,...,...,...,...,...,...,...,...,...
27845,细胞战争,"Mar 30, 2019",['DoubleC Games'],NaN,['Windows PC'],"['Indie', 'Simulator']",NaN,0,0
28027,细胞战争,"Mar 30, 2019",['DoubleC Games'],NaN,['Windows PC'],"['Indie', 'Simulator']",NaN,0,0
51496,老板攻略,"Jul 26, 2019",[],NaN,['Windows PC'],['Simulator'],NaN,0,0
21120,追忆夏色年华,"Dec 06, 2019",[],《追忆夏色年华》是由美少女游戏品牌MOONSTONE制作的一款奇幻恋爱题材作品。讲述了主人公...,['Windows PC'],['Adventure'],NaN,0,0
